Transport and Swimming Tracers
--

**v01: move to DFM without temperature.**

Don't actually have a non-temperature, wy2022 run with DWAQ output. So the machinery here
mostly copies run_dfm_bloom_tracers_v01.

Feed in Kd and insolation data.

After the run, compile maps of instantaneous growth potential, based on 
normalized vertical distribution integrated with light limitation function.

Complicating factors:
 - For buoyant tracers, depth-uniform initialization in the coastal ocean
   leads to high surface concentrations, which could then mix into the Bay.
   Practically, should be safe to assume that the time for that elevated
   concentration to be transported into the Bay is long relative to the time 
   for the vertical profile to come into balance.

A secondary run could/will create a series of releases at times of RS
scenes, and integrate until the next scene. Vertical distribution of release
is the tricky part. All at the surface? Following the distribution from the 
previous run, normalized to get match surface concentration to RS? Depth-uniform?


In [1]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")

In [4]:
import os, glob, shutil
import datetime
import six
import xarray as xr
import pandas as pd
import re

from stompy.spatial import field
from stompy import utils
from stompy.plot import plot_wkb
import xarray as xr
from scipy import ndimage

import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
from stompy.grid import unstructured_grid
from shapely import geometry
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget

In [5]:
import bloom_common

In [13]:
365*24*60

525600

In [22]:
# First, get a basic restart going
from stompy.model.delft import custom_process

class Model(custom_process.CustomProcesses,bloom_common.SFBRestartable):
    dwaq=False # will have to bring in Kd, rad after the fact.
    temperature=False
    kd_path="../Kd_2022/Kd_sentinel3_1h/Kd_sent3_20220801_20220901.nc"

    inputs_static=("/boisevol1/hpcshared/open_bay/hydro/full_res"
                   "/wy2022_r52184/sfb_dfm/inputs-static/")

    swim_speeds=[-10.0/86400.0] # positive down, m/s
    seg_function_resolution=500.0 # [m] resolution when discretizing spatiotemporal parameter to cartesian grid.
    
    def configure_general(self):
        bloom_common.configure_dfm_t141798()
                
        self.mdu['output','WaqInterval']="" # no need for DWAQ output
        self.dfm_bin_dir=os.path.join(os.environ['DELFT_SRC'],'bin')
        self.mpi_bin_dir=os.path.join(os.environ['DELFT_SRC'],'bin')

        # Some files have moved around, so up date locations
        self.mdu['output','CrsFile' ] = os.path.join(self.inputs_static, "SB-observationcrosssection.pli")
        self.mdu['output','MapInterval' ] = 3600
        
        self.mdu['geometry','LandBoundaryFile'] = os.path.join(self.inputs_static,"deltabay.ldb")
        self.mdu['geometry','FixedWeirFile'] = os.path.join(self.inputs_static,"SBlevees_tdk.pli")

        del self.mdu['waves','WaveNikuradse']

        if not self.temperature:
            self.mdu['physics','Temperature'] = 0 # and fix-up tracers below
            self.mdu['output','Wrimap_temperature'] = 0
        # for non-restart this is handled by configure(), but restart doesn't call that.
        if self.dwaq is True:                                                                                              
            self.dwaq=dwaq.WaqOnlineModel(model=self)
            
    def set_bloom_tracers(self):
        self.my_tracers=[]
        
        # I think the steps are
        #  1. add the tracer definitions to forcing data via appending/updating FlowFMold_bnd.ext
        #  2. add/overwrite the tracers in the restart file.
        
        tracers=[]

        for swim_i,swim_speed in enumerate(self.swim_speeds): # positive down, m/s
            # Names must be <=10 characters!
            conc='conc' + str(swim_i)

            def tracer_one(rst_ds,values_cell_layer):
                values_cell_layer[:,:]=1.0
            def tracer_zero(rst_ds,values_cell_layer):
                values_cell_layer[:,:] = 0.0
            tracers.append( dict(name=conc,func=tracer_one,fall_velocity_m_s=swim_speed))            

        #for tracer in tracers:
        #    # Initials don't really matter here as they are manually written to restart files.
        #    self.dwaq.substances[tracer['name']]=dwaq.Substance(initial=0)
            
        self.my_tracers=tracers
        # Adding the tracers to the ext file doesn't happen until copy_file_for_restart
        # likewise, will have to modify the restart files later.

    def update_restart_with_tracers(self):
        def modify_ic(rst_ds,**kw):
            for tracer in self.my_tracers:
                name=tracer['name']
                func=tracer['func']
                self.log.info(f"Setting tracer {name} in restart file")
                # mimic sa1 tracer
                salt=rst_ds['sa1']
                values=salt.values.copy() # ('time','nFlowElem','laydim')
                values[...] = 0.0 # don't accidentally write salt data though
                
                # updates values in place.
                func(rst_ds=rst_ds,values_cell_layer=values[0,:,:])
                rst_ds[name]=salt.dims, values
                for aname in ['coordinates','grid_mapping']:
                    if aname in salt.attrs:
                        rst_ds[name].attrs[aname]=salt.attrs[aname]
        self.modify_restart_data(modify_ic)
                
    def add_tracers_to_bcs(self):
        # take a more low-level approach compared to usual BC configuration
        # so that we can be very careful about what things change.
        ext_fn=self.mdu.filepath(('external forcing','ExtForceFile'))
        orig_ext_fn=ext_fn+".orig"
        shutil.copyfile(ext_fn,orig_ext_fn)

        bcs=self.parse_old_bc(orig_ext_fn)
        
        new_tracer_names=[t['name'] for t in self.my_tracers]
        configured_tracers={}
        
        # Try to make tracer BCs for new tracers 1 everywhere.
        # This may have to be done more manually for flow and stage BCs.
        # Note that establishing order here is very confusing. If these
        # need to be nonzero, it will take some work to know
        # it's correct. probably the strategy should be to filter out 
        # all existing BCs for these tracers, and then write them at the
        # end in our prescribed order. This [I think] is what it does
        # currently.
        
        new_bc_values=[1.0 for t in self.my_tracers]

        def name_matches(cfg_name):
            for tracer in tracers:
                if tracer['name'].lower() == cfg_name.lower():
                    if tracer['name']!=cfg_name:
                        print(f"Careful - case mismatch {cfg_name} vs {tracer['name']}")
                    return True
            return False

        
        with open(ext_fn,'wt') as fp_new:
            bc_needing_tracer=[]
            for rec in bcs:
                write_verbatim=True

                # HERE: Need to track discharge (flow) and stage BCs, then 
                # add tracer to those.
                
                quantity=rec['QUANTITY']
                if quantity.upper().startswith('INITIALTRACER'):
                    tracer_name=quantity[len("INITIALTRACER"):]
                    continue
                elif quantity.upper().startswith('TRACERBND'):
                    tracer_name=quantity[len("TRACERBND"):]
                    continue
                elif ((not self.temperature) 
                      and 
                      (quantity.upper() in ['TEMPERATUREBND','INITIALTEMPERATURE',
                                            'HUMIDITY_AIRTEMPERATURE_CLOUDINESS'])):
                    continue
                elif quantity.upper().startswith('DISCHARGE_SALINITY_TEMPERATURE_SORSIN'):
                    print("Source/sink BC entry")
                    # Yuck - have to add or remove new column(s). This only involves rewriting 
                    # the data file,though. The stanza is unchanged.
                    # Now that we drop temperature, I think orig_num_values goes from 3 to 2.
                    if self.temperature:
                        orig_num_values = 3
                    else:
                        orig_num_values = 2
                    self.add_tracer_bcs(rec,new_values=new_bc_values,orig_num_values=orig_num_values)
                elif quantity.upper() in ['DISCHARGEBND', 'WATERLEVELBND']:
                    bc_needing_tracer.append(rec)                    

                # At this point nobody ever changes the stanza, it's all written verbatim.
                if write_verbatim:
                    fp_new.write("\n".join(rec['stanza'])+"\n")
                    continue
                
            # And write out our new tracers (including ones that were skipped during 
            # transcription above
            for tracer in self.my_tracers:
                name=tracer['name']
                ic_fn=f"dummy-{name}.xyz"
                with open(os.path.join(self.run_dir,ic_fn),'wt') as fp_xyz:
                    fp_xyz.write("550000 4180000 0.0\n")
                fp_new.write("\n# NEW TRACERS\n"
                             f"QUANTITY=initialtracer{name}\n"
                             f"FILENAME={ic_fn}\n"
                             "FILETYPE=7\n"
                             "METHOD=5\n"
                             "OPERAND=O\n")
                
                if tracer['fall_velocity_m_s']!=0.0:
                    # Presumably DWAQ-based settling velocity works, too. But that would require
                    # choosing tracers that already have a settling process associated with them,
                    # or to code up a custom settling process. In contrast, if it works to 
                    # set constant settling here, where DFM handles it, things would be much simpler.
                    
                    self.log.warning("Hoping that fall velocity in can be set via DFM instead of DWAQ")
                    w=tracer['fall_velocity_m_s']
                    fp_new.write(f"TRACERFALLVELOCITY={w:.8f}\n")

                for rec in bc_needing_tracer:
                    # Copy geometry
                    base_fn=os.path.basename(rec['FILENAME'])
                    assert '.pli' in base_fn
                    # Use basename because bc_files is shared.
                    pli_fn = base_fn.replace('.pli',f"-{name}.pli")
                    shutil.copyfile(os.path.join(self.run_dir,rec['FILENAME']),
                                    os.path.join(self.run_dir,pli_fn))
                    # Fabricate data
                    tim_fn = pli_fn.replace('.pli','_0001.tim')
                    with open(os.path.join(self.run_dir,tim_fn),'wt') as fp:
                        # +-20 years around reference time. 
                        fp.write("-10000000.0 1.0\n")
                        fp.write("10000000.0 1.0\n")

                    stanza = [ 
                        f"QUANTITY=tracerbnd{name}",
                        f"FILENAME={pli_fn}",
                        "FILETYPE=9",
                        "METHOD=3",
                        "OPERAND=O"
                    ]
                    fp_new.write("\n".join(stanza) +"\n")

    def add_tracer_bcs(self,bc,new_values=[],orig_num_values=None):
        """
        Add additional columns to a source/sink data file.
        So if the new run will include two dwaq tracers, pass new_values=[0,1]
        (which would tag sources with 0 for the first and 1.0 for the second)
        orig_num_values: 3 for run with salinity and temperature. I think
        less than that if temperature and/or salinity are disabled. 
        """
        if orig_num_values is None:
            if self.temperature:
                orig_num_values=3
            else:
                orig_num_values=2
                
        # yuck...
        pli_fn=os.path.join(self.run_dir,bc['FILENAME'])
        assert pli_fn.lower().endswith('.pli')
        fn=pli_fn[:-4] + ".tim"
        assert os.path.exists(fn)
        fn_orig=fn+".orig"
        if not os.path.exists(fn_orig):
            shutil.copyfile(fn,fn_orig)
        data_orig=np.loadtxt(fn_orig)
        # drop previous forcing for new tracers. leaving time column and the original Q,S,T values
        columns=[data_orig[:,:1+orig_num_values]] 
        for new_val in new_values:
            columns.append( np.full(data_orig.shape[0],new_val))
        data=np.column_stack(columns)
        np.savetxt(fn,data,fmt="%.6g")

    def fix_ext_paths(self):
        # from run_dfm_rs_chl: Fix paths that have moved in external forcing file.
        # And also in ext boundary file. 
        ext_fn=self.mdu.filepath(('external forcing','ExtForceFile'))
        orig_ext_fn=ext_fn+".orig"
        shutil.copyfile(ext_fn,orig_ext_fn)

        print(f"Trying to fix_ext_paths in {orig_ext_fn} => {ext_fn}")
        with open(orig_ext_fn,'rt') as fp_orig:
            with open(ext_fn,'wt') as fp_new:
                for line in fp_orig:
                    m=re.match(r'\s*filename\s*=\s*([^#]+)(#.*)?',line,re.I)
                    if m:
                        ext_entry = m.group(1).strip()
                        print(f"Checking on filename {ext_entry} in external forcing file")
                        # or should it be the original run directory instead of self.run_dir?
                        real_path=os.path.abspath(os.path.join(self.run_dir,ext_entry))
                        if not os.path.exists(real_path):
                            # If it's from inputs-static replace
                            if os.path.dirname(real_path).endswith('inputs-static'):
                                real_path = os.path.join(self.inputs_static, os.path.basename(real_path))
                                assert os.path.exists(real_path)
                                line=f"FILENAME={real_path}\n"
                            else:
                                raise Exception("redirect here")
                    fp_new.write(line)

class DFMGPTracer: 
    # v00: old, dwaq based setup
    # v01: online, dfm-based setup.
    name="gp_tracers_v01"
    
    # Will start from the end of this existing run (which doesn't have the tracers)
    dfm_base_run_dir="dfm_spinup"
    
    restart_copy_names=["source_files"] # copy, because we end up modifying some

    release_time = np.datetime64("2022-08-02")
    end_timeA = np.datetime64("2022-08-24 00:00")
    #end_timeB = np.datetime64("2022-08-30")
    
    def run_to_A(self):
        """
        Find a restart point <= release time. 
        initialize and run to end_timeA. 
        """
        prev_model=Model.load(self.dfm_base_run_dir)            
        start_time=prev_model.restartable_time()
        assert start_time < self.release_time,f"Need to scan for restart time before last restart"

        # Setup a restart
        model=prev_model.create_restart(deep=True) 
        model.run_stop=self.end_timeA

        self.setup_and_run(model)

    def run_A_to_B(self):
        """
        Check for the most recent completed run ending at the release time.
        None if not found.
        Configure restart
        """
        candidates = glob.glob(self.pattern_for_run_ending_at(self.end_timeA))
        candidates.sort(reverse=True)
        for candidate in candidates: 
            if Model.run_completed(candidate):
                print(f"Will use {candidate} as previous run")
                prev_model = Model.load(candidate)
                break
        else:
            print("No completed runs end at time of release")
            return
            
        # Setup a restart
        model=prev_model.create_restart(deep=True) 
        model.run_stop=self.end_timeB
        self.setup_and_run(model)

    def setup_and_run(self,model):
        self.set_run_dir(model)
        model.configure_general()
        
        # populates self.my_tracers as a list of dictionaries
        # with ICs, names, etc.
        model.set_bloom_tracers()
            
        # This alters the MDU, so do it before write()
        model.update_restart_with_tracers()
        model.write()
        
        # Can fix some things in ext forcing file now
        model.fix_ext_paths()
        # This updates the BC data in place. Do it here so that 
        # we have a starting ext file which will be updated with
        # new tracers.
        model.add_tracers_to_bcs()
        model.partition()
        model.run_simulation()
            
    run_dir_prefix="run"
    def set_run_dir(self,model):
        start_str,stop_str=[ utils.to_datetime(t).strftime("%Y%m%dT%H%M")
                            for t in [model.run_start,model.run_stop]]
        for x in range(20):
            run_dir=os.path.join(self.name,f"{self.run_dir_prefix}_{start_str}_{stop_str}_v{x:02}")
            if not os.path.exists(run_dir): break
        else:
            raise Exception(f"Too many retries for {run_dir}")
        model.run_dir=run_dir
        model.set_restart_file() # kludge. RestartFile needs run_dir.
    def pattern_for_run_ending_at(self,end_time):
        stop_str = utils.strftime(end_time,"%Y%m%dT%H%M")
        return os.path.join(self.name,f"{self.run_dir_prefix}_*_{stop_str}_v*")
        

In [23]:
groto=DFMGPTracer()

groto.run_to_A()
#groto.run_A_to_B()

INFO:HydroModel:set_restart_file: Setting RestartFile based on self.restart_from
INFO:HydroModel:set_restart_file: Setting RestartFile based on self.restart_from
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in restart file
INFO:HydroModel:Setting tracer conc0 in res

Trying to fix_ext_paths in gp_tracers_v01/run_20220801T0000_20220824T0000_v02/FlowFMold_bnd.ext.orig => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/FlowFMold_bnd.ext
Checking on filename bc_files/MARINS1_flow.pli in external forcing file
Checking on filename bc_files/MARINS1_salt.pli in external forcing file
Checking on filename bc_files/MARINS1_temp.pli in external forcing file
Checking on filename bc_files/MARINS3_flow.pli in external forcing file
Checking on filename bc_files/MARINS3_salt.pli in external forcing file
Checking on filename bc_files/MARINS3_temp.pli in external forcing file
Checking on filename bc_files/MARINS2_flow.pli in external forcing file
Checking on filename bc_files/MARINS2_salt.pli in external forcing file
Checking on filename bc_files/MARINS2_temp.pli in external forcing file
Checking on filename bc_files/MARINN_flow.pli in external forcing file
Checking on filename bc_files/MARINN_salt.pli in external forcing file
Checking on filename bc_files/MARINN_

INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0000_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0000_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0001_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0001_net.nc


Top of partition: num_procs=16


INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0002_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0002_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0003_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0003_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0004_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0004_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0005_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0005_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0006_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0006_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0007_net.nc => gp_tracers_v01/run_20220801T0000_20220824T0000_v02/sfei_v20_0007_net.nc
INFO:HydroModel:Copying pre-partitioned grid f

About to call ['/opt/anaconda3/envs/dfm_t141798/bin/generate_parallel_mdu.sh', 'wy2022_bloom_16layer.mdu', '16', '6']


INFO:HydroModel:Running command: /opt/anaconda3/envs/dfm_t141798/bin/mpiexec -n 16 /opt/anaconda3/envs/dfm_t141798/bin/dflowfm -t 1 --autostartstop wy2022_bloom_16layer.mdu


Old Code Below Here
--

In [11]:
def hydro_name(hydro): return "wy2022_take2"

def configure_dwaq(): # May be outdated, esp. for chicago.
    # configure DWAQ:
    DELFT_SRC="/opt/software/delft/delwaq/precompiled_binaries/DFM1.6.2.49199/lnx64"
    #DELFT_SRC="/home/alliek/software/Delft3D-FM/64634"
    DELFT_SHARE=os.path.join(DELFT_SRC,"share","delft3d")
    DELFT_LIB=os.path.join(DELFT_SRC,"lib")

    os.environ['DELFT_SRC']=DELFT_SRC
    os.environ['DELFT_SHARE']=DELFT_SHARE
    
    if 'LD_LIBRARY_PATH' in os.environ:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB+":"+os.environ['LD_LIBRARY_PATH']
    else:
        os.environ['LD_LIBRARY_PATH']=DELFT_LIB

configure_dwaq()
dfm_bin=os.path.join(os.environ['DELFT_SRC'],'bin')
waqpbexport=os.path.join(dfm_bin,'waqpbexport')
waqpbimport=os.path.join(dfm_bin,'waqpbimport')

class CommonSetup(object):
    """
    Common code for various tracers runs
    """
    name='common' # should overload
    hydro=None
    base_path=None # must be set!

    force=True # whether to allow re-using an existing run
    
    # start time offset from start of hydro by this delta
    # give it some decent spinup time
    start_time=np.datetime64("2022-08-10 00:00")
    
    # set length of the run. Appears to be the end of valid output, even though the
    # hydro reports going until the 25th.
    stop_time=np.datetime64("2022-08-22 23:00") # seems that even temperature run is dicey on last step
    
    integration_option="15.60" # if set, copied to WaqModel
    time_step=3000
    map_time_step=3000 # otherwise it will default to time_step, which could be really short.    

    waq_kws={}
    def __init__(self,**kw):
        utils.set_keywords(self,kw)   
        if self.base_path is None:
            self.base_path=self.calc_base_path()
                
            yyyymmdd=utils.to_datetime(self.start_time).strftime('%Y%m%d')
            self.base_path+="_%s"%(yyyymmdd)   
            
            # And make it unique on successive runs
            for seq in range(50):
                test_path=self.base_path
                if seq>0:
                    test_path+=f"-v{seq:03}"
                if not os.path.exists(test_path):
                    self.base_path=test_path
                    break
            else:
                raise Exception("Too many runs with same name")
            
            log.info("base_path defaults to %s"%self.base_path)
    def calc_base_path(self):
        p='run_%s_%s'%(hydro_name(self.hydro),self.name)
        return p
        
    def release_conc_3d(self,*a,**kw):
        C_2d=self.release_conc_2d(*a,**kw)
        C_3d=self.hydro.extrude_element_to_segment(C_2d)
        return C_3d

    def setup_model(self):
        # Create a WaqModel, add some tracers
        self.wm=wm=dwaq.WaqModel(hydro=self.hydro,
                                 overwrite=True,
                                 base_path=self.base_path,
                                 mon_time_step=1000000, # daily
                                 time_step=self.time_step,
                                 **self.waq_kws)
        # add some option for balances.
        wm.integration_option="%s BALANCES-OLD-STYLE BAL_NOLUMPPROCESSES BAL_NOLUMPLOADS BAL_NOLUMPTRANSPORT"%self.integration_option
        #wm.start_time+= self.start_offset
        wm.start_time = self.start_time # may have to be smarter about starting on an output time step.
        # hydro reports the wrong stop time. manually set.
        if self.stop_time is not None:
            wm.stop_time=self.stop_time
        
        self.setup_tracers()
        
        wm.parameters['ACTIVE_VertDisp']=1
        wm.parameters['ScaleVDisp']=1.0 
        
    def run_waq_model(self):
        assert self.base_path is not None,"Must specify base_path"
        
        if not self.force:
            if os.path.exists(os.path.join(self.base_path,'dwaq_map.nc')):
                log.info("Run seems to exist -- will not run again")
                self.wm=dwaq.WaqModel.load(self.base_path,load_hydro=False)
                return

        self.setup_model()
        
        wm=self.wm
        wm.cmd_write_hydro()
        wm.cmd_write_inp()
        self.copy_notebook()        
        wm.cmd_delwaq1()
        wm.cmd_delwaq2()
        wm.cmd_write_nc()
    def copy_notebook(self):
        script_fn="run_transport_and_swimming_v00.ipynb"
        shutil.copyfile(script_fn,os.path.join(self.base_path,script_fn))
    def setup_tracer_continuity(self):
        # continuity tracer:
        self.wm.substances['continuity']=dwaq.Substance(initial=1.0)
        # This adds a concentration=1.0 boundary condition on all the boundaries.
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]
        self.wm.add_bc(all_bcs,'continuity',1.0)
    


Custom processes
--

This is the low-level approach. There may be some Delft tools to automate this, but
my general experience is that the tools get out of date and become unsupported.

Each process needs two pieces:
 1. Fortran code that implements the operation
 2. Tables that associate a process name, inputs, and outputs with the fortran function.
 
The same fortran code is reused for similar processes. The tables can define multiple
entries that use the same fortran implementation.

This means there is the "simple" level of customizing processes (edit the tables), and
the "complete" level (write new fortran code, along with tables to use the new fortran
subroutine).

Compiling new fortran subroutines requires using a compiler similar to what was used to
compile the original delwaq library. I think Deltares tend to use intel compilers, and
intel compilers are generally compatible with gnu compilers, so this isn't necessarily
a major hurdle (and intel compilers are readily available for free now). Nonetheless,
getting everything to work with a bespoke fortran subroutine is more error-prone than
just editing the tables.

The tables are stored three different ways, which makes the editing process at first
confusing.
 - proc_def.def and proc_def.dat: binary (NEFIS), read by delwaq at runtime
 - proces.asc: the human (for some value of human) readable table
 - a bunch of CSV files
 
You'd want to just edit proces.asc, and generate proc_def.*, but no. You have to edit
proces.asc, convert that to CSV (`waqpbimport`), and then convert back (`waqpbexport`).
Also, even though `waqpbimport` updates the CSV files, it requires them to already
exist.

You'd expect that binary distribution of delwaq would come with the CSVs, but no.


So this process is tricky with a binary compile, even if you don't want to compile.


In [25]:
# Instantaneous release, just see how the blob moves.
# Rather than using "anonymous" tracers as in the age tracer code, easier 
# to use substances that already have settling defined, but with no
# other processes. Use AlgNN tracers since there are lots of them.

class SwimmingEverywhere(CommonSetup):
    swim_speeds=[0,-5.0,-15.0,-30,-50] # positive down.
    start_time=np.datetime64("2022-08-01 00:00")
    stop_time=np.datetime64("2022-08-31 00:00") 
    kd_path="../Kd_2022/Kd_sentinel3_1h/Kd_sent3_20220801_20220901.nc"
    
    def setup_tracers(self):
        self.add_light()
        
        all_bcs=[b.decode() for b in np.unique(self.hydro.boundary_defs()['type'])]

        for swim_i,speed in enumerate(self.swim_speeds):
            #name=f'IM{swim_i+1}'
            name=f'Alg{swim_i+1:02d}'
            conc=f'BLOOM' + name
            # initial condition of 1.0
            unity=1.0
            self.wm.substances[conc]=dwaq.Substance(initial=unity)
            self.wm.parameters['VSed' + name]=  speed             
            self.wm.add_process('SED' + name)        
            
            # This adds a concentration=1.0 boundary condition on all the boundaries.
            self.wm.add_bc(all_bcs,conc,unity)

        self.wm.parameters['TaucS']=0.0 # no deposition - covers all algae.
        
    def add_light(self):
        if self.kd_path is None: return
        self.add_kd()
        self.wm.add_process(name='CalcRad')
        self.add_insolation()
        
        self.wm.map_output += ('RadSurf','Rad', 'RadBot','ExtVl')
        
    def add_insolation(self):
        cimis=xr.open_dataset('/richmondvol1/hpcshared/inputs/cimis/union_city-hourly-2022_bloom.nc')
        # Starts as PST, but the model is UTC.
        cimis=cimis.set_coords('time').swap_dims({'Date':'time'})
        cimis['time']=cimis['time']+np.timedelta64(8,'h')
        sol_rad=cimis['HlySolRad'].values
        sol_rad=utils.fill_invalid(sol_rad)

        t0=np.datetime64(self.hydro.time0)
        t_secs=((cimis.time.values-t0)/np.timedelta64(1,'s')).astype(np.int64)
        param=dwaq.ParameterTemporal(times=t_secs,values=sol_rad)
        self.wm.parameters['RadSurf'] = param
            
    def add_kd(self):
        # extrude to 3D, write seg function
        ds=xr.open_dataset(self.kd_path)
        g=unstructured_grid.UnstructuredGrid.read_ugrid(ds)
        assert np.allclose(g.cells_centroid(), self.hydro.grid().cells_centroid()),"Kd field grid does not match"

        t0=np.datetime64(self.hydro.time0)
        tsecs=(ds.time.values-t0)/np.timedelta64(1,'s')
        def seg_func(t):
            C_2d=ds['Kd'].sel(time=t0+t*np.timedelta64(1,'s'),method='nearest').values
            return self.hydro.extrude_element_to_segment(C_2d)
            
        # No self-shading, specify overall extinction directly
        self.wm.parameters['ExtVl'] = dwaq.ParameterSpatioTemporal(times=tsecs,func_t=seg_func)
        


In [26]:
if 1:
    pb=SwimmingEverywhere(hydro=hydro)

    pb.run_waq_model()

INFO:root:base_path defaults to run_wy2022_take2_common_20220801-v006
INFO:WaqModel: start time updated from hydro: 2022-05-01T00:00:00.000000
INFO:WaqModel: stop time update from hydro: 2022-10-01T00:00:00.000000
INFO:HydroFiles:Segment depth will be inferred
INFO:WaqModel:Parameters gleaned from hydro: NamedObjects([('surf', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f97fff29c40>), ('bottomdept', <stompy.model.delft.waq_scenario.ParameterSpatial object at 0x7f97fff7df70>), ('vertdisper', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2400>), ('tau', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2dc0>), ('temp', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2a60>), ('salinity', <stompy.model.delft.waq_scenario.ParameterSpatioTemporal object at 0x7f97f5ff2250>)])
INFO:WaqModel:Writing hydro data
INFO:HydroFiles:Using .bnd file, not writing out kludgey boundary-links.csv
INFO: